In [21]:
import sys
sys.path.append(r'/zhome/60/1/118435/Master_Thesis/GoalCornerDetection')
!nvidia-smi

Fri Jan  6 12:46:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:37:00.0 Off |                    0 |
| N/A   76C    P0   285W / 250W |  33936MiB / 40960MiB |    100%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:86:00.0 Off |                    0 |
| N/A   

In [23]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
from Core.DataLoader import GoalCalibrationDataset,GoalCalibrationDataset4boxes,GoalCalibrationDatasetOLD,\
train_transform,make_gt_boxes
from torchvision.models.detection import keypointrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from utils import DATA_DIR
import matplotlib.pyplot as plt
from Core.helpers import to_numpy,im_to_numpy,split_data_train_test,test_num_workers, \
find_pixelerror,eval_PCK, PCK_auc, MSE_loss_corners
import numpy as np
import cv2
import json
from Core.plottools import target_to_keypoints, visualize_results, visualize, get_prediction_images, \
crop_image, crop_images, filter_preds, kps_to_cv2, make_pred_image, visualize_cropped_results, visualize_images
import torchvision
from PIL import Image
import albumentations as A # Library for augmentations
import torchvision.transforms as T
from tqdm.notebook import tqdm
from torch.utils.data import random_split,Subset,DataLoader,RandomSampler
from Core.torchhelpers.utils import collate_fn
import pandas as pd
from pprint import pprint
from torchinfo import summary
from torch.utils.data import DataLoader
from Core.torchhelpers.utils import collate_fn
import time
import datetime

In [24]:
## Load datasets
GoalDataold = GoalCalibrationDatasetOLD(DATA_DIR)
GoalData = GoalCalibrationDataset(DATA_DIR)
GoalData4box = GoalCalibrationDataset4boxes(DATA_DIR)

All images: 854
Filtered images: 754
All images: 854
Filtered images: 754
All images: 854
Filtered images: 754
Config not given, using default values for bbox_expand.


# Scatter Plot of all keypoints

In [27]:
# save all gt keypoints in a file
for annotation_path in GoalData4box.annotation_list_filtered:
    annotation_json = json.load(open(annotation_path,'r',encoding='latin'))
    keypoints_original = np.array([[corner['CenterU'],corner['CenterV']] for corner in annotation_json['Annotations']['0'][:4]])
    print(keypoints_original)
    break

[[ 633.87382423 1183.32626849]
 [3772.06788368 1229.33404113]
 [ 555.08427035 2279.92620438]
 [3857.65795229 2241.53514016]]


# Compare GT and UA

In [ ]:
# Make dataloaders
torchloaderold = DataLoader(GoalDataold, batch_size=2,collate_fn=collate_fn)
torchloader = DataLoader(GoalData, batch_size=2,collate_fn=collate_fn)

In [3]:
#%% load annotation points

keypoints_dict = {target['image_id']: target['keypoints']
for _,targets in tqdm(torchloader)
for target in targets}

#%% load old annotation points
keypointsold_dict = {target['image_id']: target['keypoints']
for _,targets in tqdm(torchloaderold)
for target in targets}

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 377/377 [04:33<00:00,  1.38it/s]


In [ ]:
#%% Do comparison of old keypoints and new keypoints
distances = [np.linalg.norm(kp[:2]-kp_old[:2])
            for (idold,kpsold),(id,kps) in tqdm(zip(keypointsold_dict.items(),keypoints_dict.items()))
            for obj_old,obj in zip(kpsold,kps)
            for kp_old, kp in zip(obj_old,obj)]

distances_TL = distances[0::4]
distances_TR = distances[1::4]
distances_BL = distances[2::4]
distances_BR = distances[3::4]

#%%
mean_all = np.mean(distances)
mean_TL = np.mean(distances_TL)
mean_TR = np.mean(distances_TR)
mean_BL = np.mean(distances_BL)
mean_BR = np.mean(distances_BR)

print(f'mean all: {mean_all}')
print(f'mean TL: {mean_TL}')
print(f'mean TR: {mean_TR}')
print(f'mean BL: {mean_BL}')
print(f'mean BR: {mean_BR}')

# outputs from running above
# mean all: 37.15945816040039
# mean TL: 23.0086669921875
# mean TR: 45.796661376953125
# mean BL: 42.918609619140625
# mean BR: 36.91389465332031

In [5]:
GT_loader = torchloader
UA_loader = torchloaderold

In [6]:

num_objects = 1
pixelerrors_all = []
print(f'Finding pixelerror for all predictions...')
start_time = time.time()
# Run through all images and get the pixel distance (error) between predictions and ground-truth
for (_,targetsGT),(_,targetsUA) in tqdm(zip(GT_loader, UA_loader)):
    # move outputs to cpu
    # outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
    # extract the euclidean distance error (in pixels) between every ground-truth and detection keypoint in the batch. Also return image_ids and labels for every distance measure for reference
    for targetGT, targetUA in zip(targetsGT, targetsUA):
        label_to_gts = {}
        label_to_dts = {}
        # make a dictionary for the gts and dts in every target and output that save the label, keypoints and scores, for later sorting
        for labelGT,kpGT,labelUA,kpUA in zip(targetGT['labels'],targetGT['keypoints'],targetUA['labels'],targetUA['keypoints']):
            label_to_gts[labelGT.item()] = kpGT
            label_to_dts[labelUA.item()] = kpUA
        # compare the gt and dt of every object with the same label, taking only the highest scored one
        for label in range(1,num_objects+1):
            # get the obj_gt and obj_dt for this label (obj_dt may not exist)
            obj_gt = label_to_gts[label]
            obj_dt = label_to_dts[label]
            # find the distance between every gt and gt for this label, and add to list of distances, along witht the image_id
            for gt,dt in zip(obj_gt,obj_dt):
                pixelerrors_all.append((targetUA['image_id'].item(), label, np.linalg.norm(dt[:2]-gt[:2])))
num_keypoints = 4
pixelerrors_TL = pixelerrors_all[0::num_keypoints]
pixelerrors_TR = pixelerrors_all[1::num_keypoints]
pixelerrors_BL = pixelerrors_all[2::num_keypoints]
pixelerrors_BR = pixelerrors_all[3::num_keypoints]
pixelerrors = {
    "all":pixelerrors_all,
    "TL":pixelerrors_TL,
    "TR":pixelerrors_TR,
    "BL": pixelerrors_BL,
    "BR": pixelerrors_BR
    }
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print(f'Total time: {total_time_str}')

Finding pixelerror for all predictions...


377it [09:03,  1.44s/it]

Total time: 0:09:03


In [7]:
pixelerrors

{'all': [(tensor([0]), 1, 2.6891737),
  (tensor([0]), 1, 5.13185),
  (tensor([0]), 1, 1.042285),
  (tensor([0]), 1, 4.5229607),
  (tensor([1]), 1, 9.246991),
  (tensor([1]), 1, 7.875417),
  (tensor([1]), 1, 8.991363),
  (tensor([1]), 1, 22.02483),
  (tensor([2]), 1, 0.0),
  (tensor([2]), 1, 6.5070934),
  (tensor([2]), 1, 4.6805663),
  (tensor([2]), 1, 2.4167585),
  (tensor([3]), 1, 5.046522),
  (tensor([3]), 1, 6.363469),
  (tensor([3]), 1, 0.967041),
  (tensor([3]), 1, 6.040187),
  (tensor([4]), 1, 3.2586095),
  (tensor([4]), 1, 1.1447148),
  (tensor([4]), 1, 1.943025),
  (tensor([4]), 1, 1.5268555),
  (tensor([5]), 1, 3.9681618),
  (tensor([5]), 1, 2.1148589),
  (tensor([5]), 1, 18.152758),
  (tensor([5]), 1, 0.0),
  (tensor([6]), 1, 11.614764),
  (tensor([6]), 1, 3.9374247),
  (tensor([6]), 1, 13.389271),
  (tensor([6]), 1, 31.22655),
  (tensor([7]), 1, 2.5255785),
  (tensor([7]), 1, 2.0967333),
  (tensor([7]), 1, 2.1520035),
  (tensor([7]), 1, 4.107664),
  (tensor([8]), 1, 7.538458

In [18]:
pixelerrors_filtered = {cat:[metric for metric in metrics if metric[2] < 500]
                       for cat,metrics in pixelerrors.items()}

In [8]:
print('Making summary statistics of errors for outliers table...')
data = []
for cat,metrics_og in pixelerrors.items():
    # filter missing elements (with value None) out of metrics (list of tuples)
    metrics = [metric for metric in metrics_og if metric[2] is not None]
    if len(metrics) > 0:
        # extracts the errors list and discards the image ids and labels
        _,_,errors = zip(*metrics)
    else:
        print(f'category: {cat} did not have any elements, skipping...')
        continue
    # find the missing predictions, i.e. when a corner is not predicted at all in an image
    missing_tuplist = [(image_id,label,error) for image_id,label,error in metrics_og if error is None]
    num_missing_preds = len(missing_tuplist)
    if num_missing_preds > 0:
        missing_ids, missing_labels, _ = zip(*missing_tuplist)
        # filter duplicate missing_ids, but keep all missing_labels that way we don't get duplicate missing_predims
        missing_ids = tuple(set(missing_ids))
    else:
        missing_ids, missing_labels = (),()
    Ndata = len(errors)
    minval = np.min(errors)
    maxval = np.max(errors)
    std = np.std(errors)
    mean = np.mean(errors)
    MSE = np.mean(np.array(errors)**2)
    median = np.median(errors)
    # make sure the inliner_min doesn't go below 0
    inlier_min = np.maximum(mean-3*std,0)
    inlier_max = mean+3*std
    outliers_tuplist = [(image_id,label,error) for image_id,label,error in metrics if not inlier_min <= error <= inlier_max]
    num_outliers = len(outliers_tuplist)
    pct_outliers = num_outliers / Ndata
    if num_outliers > 0:
        outlier_ids, outlier_labels, outliers = zip(*outliers_tuplist)
        # filter duplicate outlier_ids, but keep all outlier_labels and outliers (error values) that way we don't get duplicate outlier_predims
        outlier_ids = tuple(set(outlier_ids))
    else:
        outliers, outlier_ids, outlier_labels = (),(),()
    data.append((cat, Ndata, minval, maxval, std, mean, MSE, median, inlier_min, inlier_max, num_outliers, pct_outliers, outliers, outlier_ids, outlier_labels, num_missing_preds, missing_ids, missing_labels))
table = pd.DataFrame(data, columns =['cat', 'Ndata', 'min', 'max', 'std', 'mean', 'MSE', 'median', 'inlier_min', 'inlier_max', '#outliers', '%outliers', 'outliers', 'outlier_ids', 'outlier_labels', '#missing_corners', 'missing_ids', 'missing_labels'])
print('done')

Making summary statistics of errors for outliers table...


In [9]:
table

,cat,Ndata,min,max,std,mean,MSE,median,inlier_min,inlier_max,#outliers,%outliers,outliers,outlier_ids,outlier_labels,#missing_corners,missing_ids,missing_labels
0,all,3016,0.0,3351.385010,229.491501,38.127964,54120.085938,7.042583,0.0,726.602467,41,0.013594,"(2352.139, 898.32574, 2610.3577, 1868.8047, 18...","([tensor(108)], [tensor(104)], [tensor(162)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,(),()
1,TL,754,0.0,1868.804688,122.360283,23.003927,15501.217773,6.671722,0.0,390.084776,9,0.011936,"(1868.8047, 634.35876, 664.5813, 628.79004, 15...","([tensor(248)], [tensor(254)], [tensor(104)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1)",0,(),()
2,TR,754,0.0,2352.138916,273.637756,47.310852,77115.929688,5.919717,0.0,868.224121,15,0.019894,"(2352.139, 1817.0251, 1978.9193, 2065.2021, 17...","([tensor(108)], [tensor(104)], [tensor(162)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",0,(),()
3,BL,754,0.0,3351.385010,246.806931,45.217602,62958.292969,8.605177,0.0,785.638393,14,0.018568,"(898.32574, 1894.3959, 2044.5234, 2155.6711, 8...","([tensor(162)], [tensor(104)], [tensor(194)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",0,(),()
4,BR,754,0.0,2611.705078,244.002945,36.979473,60904.914062,7.342476,0.0,768.988308,9,0.011936,"(2610.3577, 1781.6359, 1841.2131, 2610.7034, 2...","([tensor(108)], [tensor(194)], [tensor(162)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1)",0,(),()


In [19]:
print('Making summary statistics of errors for outliers table...')
data = []
for cat,metrics_og in pixelerrors_filtered.items():
    # filter missing elements (with value None) out of metrics (list of tuples)
    metrics = [metric for metric in metrics_og if metric[2] is not None]
    if len(metrics) > 0:
        # extracts the errors list and discards the image ids and labels
        _,_,errors = zip(*metrics)
    else:
        print(f'category: {cat} did not have any elements, skipping...')
        continue
    # find the missing predictions, i.e. when a corner is not predicted at all in an image
    missing_tuplist = [(image_id,label,error) for image_id,label,error in metrics_og if error is None]
    num_missing_preds = len(missing_tuplist)
    if num_missing_preds > 0:
        missing_ids, missing_labels, _ = zip(*missing_tuplist)
        # filter duplicate missing_ids, but keep all missing_labels that way we don't get duplicate missing_predims
        missing_ids = tuple(set(missing_ids))
    else:
        missing_ids, missing_labels = (),()
    Ndata = len(errors)
    minval = np.min(errors)
    maxval = np.max(errors)
    std = np.std(errors)
    mean = np.mean(errors)
    MSE = np.mean(np.array(errors)**2)
    median = np.median(errors)
    # make sure the inliner_min doesn't go below 0
    inlier_min = np.maximum(mean-3*std,0)
    inlier_max = mean+3*std
    outliers_tuplist = [(image_id,label,error) for image_id,label,error in metrics if not inlier_min <= error <= inlier_max]
    num_outliers = len(outliers_tuplist)
    pct_outliers = num_outliers / Ndata
    if num_outliers > 0:
        outlier_ids, outlier_labels, outliers = zip(*outliers_tuplist)
        # filter duplicate outlier_ids, but keep all outlier_labels and outliers (error values) that way we don't get duplicate outlier_predims
        outlier_ids = tuple(set(outlier_ids))
    else:
        outliers, outlier_ids, outlier_labels = (),(),()
    data.append((cat, Ndata, minval, maxval, std, mean, MSE, median, inlier_min, inlier_max, num_outliers, pct_outliers, outliers, outlier_ids, outlier_labels, num_missing_preds, missing_ids, missing_labels))
table = pd.DataFrame(data, columns =['cat', 'Ndata', 'min', 'max', 'std', 'mean', 'MSE', 'median', 'inlier_min', 'inlier_max', '#outliers', '%outliers', 'outliers', 'outlier_ids', 'outlier_labels', '#missing_corners', 'missing_ids', 'missing_labels'])
print('done')

Making summary statistics of errors for outliers table...
done


In [20]:
table

,cat,Ndata,min,max,std,mean,MSE,median,inlier_min,inlier_max,#outliers,%outliers,outliers,outlier_ids,outlier_labels,#missing_corners,missing_ids,missing_labels
0,all,2969,0.0,452.307861,15.579991,11.141775,366.875305,6.904842,0.0,57.881749,40,0.013473,"(60.80888, 114.06596, 59.862465, 59.39486, 60....","([tensor(326)], [tensor(346)], [tensor(528)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,(),()
1,TL,745,0.0,263.227112,17.262838,11.375406,427.405487,6.586886,0.0,63.163921,11,0.014765,"(114.06596, 263.2271, 115.24638, 111.86964, 80...","([tensor(108)], [tensor(346)], [tensor(194)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",0,(),()
2,TR,739,0.0,62.930481,9.358284,8.761101,164.334366,5.805455,0.0,36.835953,22,0.029770,"(44.98065, 41.817093, 41.462845, 46.751484, 40...","([tensor(326)], [tensor(528)], [tensor(676)], ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,(),()
3,BL,740,0.0,452.307861,21.406685,13.670074,645.117004,8.431573,0.0,77.890128,4,0.005405,"(452.30786, 117.612595, 94.12312, 98.78012)","([tensor(108)], [tensor(564)], [tensor(574)], ...","(1, 1, 1, 1)",0,(),()
4,BR,745,0.0,69.720505,10.730296,10.758314,230.880585,7.227025,0.0,42.949203,18,0.024161,"(46.597557, 43.83001, 49.64034, 61.822186, 51....","([tensor(57)], [tensor(275)], [tensor(152)], [...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,(),()
